한국어 영화 리뷰 - BERT

In [1]:
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)

In [2]:
#rating 6이상이면, 긍정 라벨 생성 y로 저장
import numpy as np
from sklearn.model_selection import train_test_split

# x(review컬럼),y  데이터분할 데이터 개수 확인
df.head()
y=np.array([1 if data > 6 else 0 for data in df.rating]) #X가 numpy 이므로 numpy array 맞춰주기
X=df.review.values
# 데이터셋을 학습 검증 평가로 나눈다. X_train, X_val, X_test
X_, X_test, y_, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_,y_, test_size=0.2, stratify=y_, random_state=42)
print (f'X_test {len(X_test)}, y_test {len(y_test)},\nX_train {len(X_train)}, y_train {len(y_train)}, \nX_val {len(X_val)}, y_val {len(y_val)}')
print(X_train.shape, X_val.shape, X_test.shape)

X_test 2945, y_test 2945,
X_train 9424, y_train 9424, 
X_val 2356, y_val 2356
(9424,) (2356,) (2945,)


In [ ]:
# %pip install evaluate
%pip install datasets

In [4]:
import torch
# from datasets import load_metric   #2022년 이후로 Hugging Face에서 deprecated --> evaluate 변경
                                    # old system --> legacy system
import evaluate
metric = evaluate.load('accuracy')



In [5]:
from huggingface_hub import list_organization_members
def compute_metrics (eval_pred):
    '''
    Args:
        eval_pred : logits.labels를 가지고 있는 dataset
    Returns:
        accuracy
    '''
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
# 데이터셋 생성 클래스 (상속)
# __init__ __getitem__ __len__
# X,y  각각 텐서 y의 개수
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        # BERT encoding 되면 input_ids, token_type_ids, attention_mask --> 딕셔너리 --> torch.tensor.. 왜?
        item = {key : torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [7]:
# BERT 토크나이저 한국어 가져오기
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased') #다국어 지원(한국어)
print(tokenizer.tokenize('안녕하세요, 반갑습니다'))  #단순 토커나이저에 넣어봄.
inputs = tokenizer("안녕하세요. 반갑습니다") #모델에 넣어서 확인해봄
print(inputs)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

c:\Users\playdata2\miniconda3\envs\P10\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\playdata2\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

['안', '##녕', '##하', '##세', '##요', ',', '반', '##갑', '##습', '##니다']
{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
# 2022년 이후로는 Auto~~ 토크나이저와 모델을 사용하도록 권장(벤더사에서 업데이트시 유리)
# Auto - 모델에 대한 토커나이저가 자동으로 가져옴.
# --> Auto 쓰면 BERT fast를 자동으로 가져옴

# 그래서... 앞에꺼 다시 작성해보면.. 결과는 같지만, 이 방식이 최근방식. AutoTokenizer!!
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
print(tokenizer.tokenize('안녕하세요, 반갑습니다'))
inputs = tokenizer("안녕하세요. 반갑습니다") #모델에 넣어서 확인해봄
print(inputs)

['안', '##녕', '##하', '##세', '##요', ',', '반', '##갑', '##습', '##니다']
{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
# nBERT + Trainer로 미세조정(Fine Tuning)
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# 토큰화 (input = encoding )
# tolist() : X_train, X_val, X_test --> array 이므로 list화 시킴
# return_tensors='pt' 학습하려면 tensor 형태 여야함
# 파라미터 max_length를 지정하지않으면, 가장 긴 문장을 기준으로 함.
train_input = tokenizer(X_train.tolist(), truncation=True, padding=True, return_tensors='pt')
val_input = tokenizer(X_val.tolist(), truncation=True, padding=True, return_tensors='pt')
test_input = tokenizer(X_test.tolist(), truncation=True, padding=True, return_tensors='pt')


# Dataset 생성
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)


# 분류모델 생성
model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)


# Trainer에 사용할 Argumnet 셋팅
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    eval_strategy="steps",
    eval_steps=500,              #
    report_to='none'            #
)


# Trainer 객체 생성 및 학습(미세조정)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    )


trainer.train()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\playdata2\miniconda3\envs\P10\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\playdata2\AppData\Local\Temp\ipykernel_16480\4202792175.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key : torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss


In [ ]:
# 평가
trainer.evaluate(test_dataset)